In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlsp-padded/reshaped/x_train548.npy
/kaggle/input/mlsp-padded/reshaped/x_train224.npy
/kaggle/input/mlsp-padded/reshaped/x_train768.npy
/kaggle/input/mlsp-padded/reshaped/x_train607.npy
/kaggle/input/mlsp-padded/reshaped/x_train1.npy
/kaggle/input/mlsp-padded/reshaped/x_train60.npy
/kaggle/input/mlsp-padded/reshaped/x_train945.npy
/kaggle/input/mlsp-padded/reshaped/x_train158.npy
/kaggle/input/mlsp-padded/reshaped/x_train706.npy
/kaggle/input/mlsp-padded/reshaped/x_train428.npy
/kaggle/input/mlsp-padded/reshaped/x_train229.npy
/kaggle/input/mlsp-padded/reshaped/x_train305.npy
/kaggle/input/mlsp-padded/reshaped/x_train578.npy
/kaggle/input/mlsp-padded/reshaped/x_train990.npy
/kaggle/input/mlsp-padded/reshaped/x_train950.npy
/kaggle/input/mlsp-padded/reshaped/x_train366.npy
/kaggle/input/mlsp-padded/reshaped/x_train605.npy
/kaggle/input/mlsp-padded/reshaped/x_train854.npy
/kaggle/input/mlsp-padded/reshaped/x_train620.npy
/kaggle/input/mlsp-padded/reshaped/x_train290.npy
/ka

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import json
import librosa
    
def get_data_features(n_mfcc):
    lst_x = []
    lst_y = list(np.load("../input/mlsp-padded/reshaped/y_train.npy"))
    with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
        dict1 = json.loads(f.read())
    for m in range(1000):
        z = np.load("../input/mlsp-padded/reshaped/x_train" + str(m) + ".npy")
        features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 16)
        lst_x.append(features)
    return lst_x, lst_y, dict1

def train_val_split(lst1, lst_y): # input shapes are: (1000, 128, 2584) and (1000, 10)
    count = 1
    lstx_train = []
    lstx_test = []
    lsty_train = []
    lsty_test = []
    for m in range(len(lst1)):
        if m <= (count*100) - 21:
            lstx_train.append(lst1[m])
            lsty_train.append(lst_y[m])
        else:
            lstx_test.append(lst1[m])
            lsty_test.append(lst_y[m])
        if m > (count*100) - 2:
            count = count + 1
    X_train = np.array(lstx_train)
    y_train = np.array(lsty_train)
    X_test = np.array(lstx_test)
    y_test =  np.array(lsty_test)
    del(lst1)
    x_train, y_train = shuffle(X_train, y_train)
    x_test, y_test = shuffle(X_test, y_test)
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    return x_train, x_test, y_train, y_test

In [3]:
n_mfcc = 16
x_train, y_train, dict1 = get_data_features(n_mfcc)

In [4]:
x_train, x_test, y_train, y_test = train_val_split(x_train, y_train)

In [5]:
print(dict1)
(temp, f, t, num) = x_train.shape
print(f)
print(t)
print(num)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

{'Bark': 0, 'Meow': 1, 'Siren': 2, 'Shatter': 3, 'Knock': 4, 'Crying_and_sobbing': 5, 'Microwave_oven': 6, 'Vehicle_horn_and_car_horn_and_honking': 7, 'Doorbell': 8, 'Walk_and_footsteps': 9}
16
2584
1
(800, 16, 2584, 1)
(200, 16, 2584, 1)
(800, 10)
(200, 10)


In [8]:
N_classes = 10
def make_model(f, t, num):
    model1 = tf.keras.models.Sequential()
    model1.add(tf.keras.layers.Input(shape=(f, t, num)))
    model1.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.Flatten())
    model1.add(tf.keras.layers.Dense(256, activation='leaky_relu'))
    #model1.add(tf.keras.layers.Dense(128, activation='relu'))
    model1.add(tf.keras.layers.Dense(64, activation='leaky_relu'))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.Dropout(0.5))
    model1.add(tf.keras.layers.Dense(N_classes, activation='softmax'))
    return model1
model1 = make_model(f, t, num)

In [9]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 8, 1292, 32)       320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 1292, 32)       128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 646, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 4, 646, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 646, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 4, 646, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 323, 64)       

In [10]:
model1.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

# Train model for 50 epochs, capture the history
history = model1.fit(x_train, y_train, verbose = 1, epochs=50, validation_data=(x_test, y_test))

2022-10-12 01:38:47.933701: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-10-12 01:38:50.000446: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


25/25 [==============================] - 9s 44ms/step - loss: 2.6132 - precision: 0.3465 - recall: 0.1100 - val_loss: 12.1438 - val_precision: 0.1907 - val_recall: 0.1850
Epoch 2/50
25/25 [==============================] - 0s 16ms/step - loss: 1.6937 - precision: 0.6011 - recall: 0.2675 - val_loss: 5.4571 - val_precision: 0.1720 - val_recall: 0.1600
Epoch 3/50
25/25 [==============================] - 0s 17ms/step - loss: 1.2475 - precision: 0.7396 - recall: 0.4225 - val_loss: 3.5045 - val_precision: 0.5000 - val_recall: 0.1200
Epoch 4/50
25/25 [==============================] - 0s 16ms/step - loss: 0.8793 - precision: 0.8595 - recall: 0.5813 - val_loss: 2.7459 - val_precision: 0.2418 - val_recall: 0.1850
Epoch 5/50
25/25 [==============================] - 0s 16ms/step - loss: 0.6495 - precision: 0.8838 - recall: 0.6938 - val_loss: 1.9246 - val_precision: 0.5455 - val_recall: 0.2700
Epoch 6/50
25/25 [==============================] - 0s 16ms/step - loss: 0.4684 - precision: 0.9184 - rec

In [11]:
model1.save('model_3.h5')

In [12]:
yhat_probs = model1.predict(x_test, verbose=1)
# predict crisp classes for test set
#yhat_classes = model.predict_classes(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
y_target = []
for m in y_test:
    r = np.where(m == 1)
    y_target.append(int(r[0]))
    #y_target.append()

7/7 [==============================] - 0s 4ms/step
[6 7 9 9 5 3 0 5 9 9 5 4 7 9 4 8 7 7 0 2 4 2 8 4 3 9 4 6 0 8 5 4 8 6 7 0 3
 3 4 1 6 7 7 5 0 8 7 2 6 5 3 8 9 6 9 8 1 5 5 8 6 0 2 8 1 2 2 6 5 4 3 6 6 9
 8 4 7 6 2 7 3 2 0 3 9 8 6 1 7 8 9 1 7 3 2 5 3 1 7 4 1 8 6 0 5 5 8 2 7 1 4
 3 6 6 6 2 5 5 7 3 0 7 8 1 6 1 8 4 7 8 8 4 3 4 2 3 0 0 7 2 5 4 7 8 2 7 5 5
 8 9 1 7 7 7 0 2 0 6 7 1 6 7 9 8 0 7 4 2 2 1 2 3 1 4 5 9 5 3 2 2 1 6 4 5 1
 3 6 0 2 0 3 7 2 1 7 6 9 5 9 7]


In [13]:
print(len(y_hat))
print(len(y_target))

200
200


In [14]:
from sklearn.metrics import classification_report
report = classification_report(y_target, y_hat, output_dict=True)
#report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [15]:
lst = []
for m in range(10):
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)
lst.append("accuracy")
lst.append("macro avg")
lst.append("weighted avg")
report_df["Labels"] = lst

In [16]:
report_df

,precision,recall,f1-score,support,Labels
0,0.875000,0.70,0.777778,20.00,Bark
1,0.823529,0.70,0.756757,20.00,Meow
2,0.818182,0.90,0.857143,20.00,Siren
3,0.777778,0.70,0.736842,20.00,Shatter
4,0.888889,0.80,0.842105,20.00,Knock
5,0.666667,0.70,0.682927,20.00,Crying_and_sobbing
6,0.818182,0.90,0.857143,20.00,Microwave_oven
7,0.655172,0.95,0.775510,20.00,Vehicle_horn_and_car_horn_and_honking
8,0.857143,0.90,0.878049,20.00,Doorbell
9,0.687500,0.55,0.611111,20.00,Walk_and_footsteps


In [17]:
report_df.to_csv("model_3_result.csv")